## КТ12
### Генерация gRPC

In [1]:
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. registraion.proto

In [2]:
!python -m grpc_tools.protoc -I. --python_out=. --grpc_python_out=. database.proto

### gRPC-сервисы

In [63]:
%%file reg_service.py
import grpc
import registration_pb2
import registration_pb2_grpc
import database_pb2


class RegistrationServiceServicer(registration_pb2_grpc.RegistrationServiceServicer):
    def __init__(self, database_stub):
        self.database_stub = database_stub

    def RegisterUser(self, request, context):
        username = request.username

        if not username:
            return registration_pb2.RegisterResponse(success=False, message="Username is required")

        check_request = database_pb2.CheckUserRequest(username=username)
        try:
            response = self.database_stub.CheckUserExists(check_request)

            if response.exists:
                return registration_pb2.RegisterResponse(success=False, message="User already exists")
            else:
                return registration_pb2.RegisterResponse(success=True, message="User registered successfully")
        except grpc.RpcError as e:
            return registration_pb2.RegisterResponse(success=False, message=f"Database error: {e}")

Overwriting reg_service.py


In [66]:
%%file db_service.py
import database_pb2
import database_pb2_grpc


class DatabaseServiceServicer(database_pb2_grpc.DatabaseServiceServicer):
    def __init__(self):
        self.users = set()

    def CheckUserExists(self, request, context):
        username = request.username
        exists = username in self.users

        if not exists:
            self.users.add(username)

        return database_pb2.CheckUserResponse(exists=exists)

Overwriting db_service.py


### Тесты

In [64]:
%%file test_grpc.py
import pytest
import grpc
import registration_pb2
import registration_pb2_grpc
import database_pb2
import database_pb2_grpc
from concurrent import futures
from db_service import DatabaseServiceServicer
from reg_service import RegistrationServiceServicer


def create_channel(host, port):
    return grpc.insecure_channel(f"{host}:{port}")


@pytest.fixture(scope="session")
def database_service():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    database_pb2_grpc.add_DatabaseServiceServicer_to_server(DatabaseServiceServicer(), server)

    server.add_insecure_port("[::]:50051")
    server.start()
    yield server
    server.stop(grace=None)


@pytest.fixture(scope="session")
def registration_service(database_service):
    channel = create_channel("localhost", 50051)
    database_stub = database_pb2_grpc.DatabaseServiceStub(channel)
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
    registration_pb2_grpc.add_RegistrationServiceServicer_to_server(RegistrationServiceServicer(database_stub), server)

    server.add_insecure_port("[::]:50052")
    server.start()
    yield server
    server.stop(grace=None)


def test_registration(registration_service):
    channel = create_channel("localhost", 50052)
    stub = registration_pb2_grpc.RegistrationServiceStub(channel)
    request = registration_pb2.RegisterRequest(username="test", password="123")

    response = stub.RegisterUser(request)
    assert response.success


def test_registration_empty_username(registration_service):
    channel = create_channel("localhost", 50052)
    stub = registration_pb2_grpc.RegistrationServiceStub(channel)
    request = registration_pb2.RegisterRequest(username="", password="123")

    response = stub.RegisterUser(request)
    assert response.success is False


def test_registration_existing_user(database_service):
    channel = create_channel("localhost", 50051)
    stub = database_pb2_grpc.DatabaseServiceStub(channel)
    request = database_pb2.CheckUserRequest(username="test")

    response = stub.CheckUserExists(request)
    assert response.exists


def test_registration_non_existing_user(database_service):
    channel = create_channel("localhost", 50051)
    stub = database_pb2_grpc.DatabaseServiceStub(channel)
    request = database_pb2.CheckUserRequest(username="idontexist")
    response = stub.CheckUserExists(request)
    assert response.exists is False


def test_integration(registration_service, database_service):
    channel_db = create_channel("localhost", 50051)
    db_stub = database_pb2_grpc.DatabaseServiceStub(channel_db)
    db_request = database_pb2.CheckUserRequest(username="test")
    db_stub.CheckUserExists(db_request)

    channel_reg = create_channel("localhost", 50052)
    reg_stub = registration_pb2_grpc.RegistrationServiceStub(channel_reg)
    reg_request = registration_pb2.RegisterRequest(username="test", password="123")

    response = reg_stub.RegisterUser(reg_request)
    assert response.success is False

Overwriting test_grpc.py


In [65]:
!pytest test_grpc.py

============================= test session starts =============================
platform win32 -- Python 3.9.7, pytest-8.3.4, pluggy-1.5.0
rootdir: C:\Users\acva0\Projects\PycharmProjects\WebTesting\KT12
plugins: allure-pytest-2.13.5, anyio-4.8.0, variables-3.1.0
collected 5 items

test_grpc.py .....                                                       [100%]

============================== 5 passed in 0.38s ==============================
